In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
# Import our input dataset
learning_data_df = pd.read_csv('../Resources/nhes_19_pfi_cleaned.csv')
learning_data_df.head()

,Unnamed: 0,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,...,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,0
1,4,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,1
2,7,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
3,9,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,1
4,11,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0


In [6]:
# create features
X = learning_data_df.drop(['SEGRADES_0', 'Unnamed: 0','SEGRADES_1'], axis=1)

# create target
y = learning_data_df['SEGRADES_1']

In [7]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net
number_input_features = len(X_train)
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1253000   
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,273,201
Trainable params: 1,273,201
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100


ValueError: in user code:

    File "C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_com

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")